## Finetuning openLlama using PEFT+Lora

In [1]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [2]:
!nvidia-smi

Thu May 11 02:08:01 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    47W / 400W |      3MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# !pip install langchain==0.0.142

In [ ]:
# !pip install llama-cpp-python


In [ ]:
# !git clone https://github.com/tloen/alpaca-lora.git

In [ ]:
# %cd alpaca-lora/

/content/alpaca-lora


In [ ]:
# %cd ..


In [ ]:
!pip install -q datasets loralib sentencepiece

In [ ]:
# !pip install -q bitsandbytes datasets accelerate loralib
# !pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

In [23]:
# !pip uninstall transformers --yes
# !pip install -q git+https://github.com/zphang/transformers@c3dc391
# !pip install git+https://github.com/zphang/transformers.git@llama_push

!pip install -q git+https://github.com/huggingface/peft.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 12.0 MB/s eta 0:00:00


In [ ]:
!git lfs install
!git clone https://huggingface.co/openlm-research/open_llama_7b_preview_300bt

In [ ]:
!pip install openai

In [ ]:
!pip install spacy

In [ ]:
!pip install spacy-transformers

In [ ]:
! spacy download en_core_web_trf

In [ ]:
!pip install tiktoken

In [ ]:
!pip install bitsandbytes

In [ ]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:           83Gi       1.1Gi        73Gi       6.0Mi       9.2Gi        81Gi
Swap:            0B          0B          0B


In [11]:
template = """Your task is to extract the person's name from the sentence delimited by triple backticks
along with their details mentioned below:
1. their designation, 
2. their companies ,
3. Number or percentage of shares bought
4. Type of shares
5. acquistions

The person should be either:
1. A person involved in sell or buying of any company's share at a large scale.
2. A person who is equivalent to CEO, chairman, board of directors,founder etc. 

If person name is not present then do not extract anything and respond 'None'
Extract the data in the list of python dictionaries format.
If the values are not available mark it as 'NA'.


sentence : ```{text}```
"""
template1 = """
Your task is to extract the following details from the sentence delimited by triple backticks:

1.Name (if available)
2.Designation (if available)
3.Company (if available)
4.Net Worth (if available)
5.Shares bought
6.Shares sold
7.Type of Shares (if available)
8.Acquisitions by the Company (if available)
Extract the data in a list of Python dictionaries format. If the values are not available, mark them as 'NA'.
The person whose details are to be extracted should meet any or all of the following criteria:

1.Involved in the buying or selling of any company's shares on a large scale.
2.Has a designation equivalent to CEO, chairman, board of directors, founder, etc.
3.Has a high net worth as indicated by indicators such as business ownership or executive leadership, 
real estate holdings, philanthropy, luxury lifestyle, or a diversified investment portfolio.

Exclude individuals who have job titles or roles that are not typically associated with high net worth, such 
as "Research Analyst", "Marketing Manager", etc.

If the person's name is not present, but the designation, company, and net worth are mentioned, extract those details. 
If none of the details are available, respond with 'None'.

The sentence to be analyzed is:
```{text}```
"""

template2 = """
Your task is to extract the following details from the sentence delimited by triple backticks:

1. Company from where shares were sold
2. Company from which shares were bought
3. Company of person who bought the share
4. Number of shares sold
5. Price at which shares were sold
6. Number of shares bought
7. Price at which shares were bought
8. Type of Shares (if available)

Extract the data in a list of Python dictionaries format. If the values are not available, mark them as 'NA'.

The sentence to be analyzed should mention the sale or purchase of shares by a company or individual. 
If none of the details are available, respond with 'None'.

The sentence to be analyzed is:

```{text}```

"""

template3 = """Extract details of individuals 
involved in buying/selling shares at large scale, with name, designation, company, net worth, shares bought/sold, type of shares, and acquisitions. 
Exclude low net worth titles in list of dictionaries. If details not available, mark as NA. Use input as the sentence from where the aforementioned information needs to be extracted"""

In [13]:
import pandas as pd 
import json , os ,re

## Finetuning LLM llama

In [14]:
df = pd.read_excel(r"/content/check.xlsx")

In [15]:
temp = []
for idx, row in df.iterrows():
  x = {"instruction":template1,"input":row['sentence'],"output":row['extraction1']}
  temp.append(x)


In [16]:
json_object = json.dumps(temp)

In [18]:
with open("data.json","w") as f:
  f.write(json_object)

In [ ]:
from datasets import load_dataset
# from transformers import LLaMATokenizer


# tokenizer = LLaMATokenizer.from_pretrained("decapoda-research/llama-7b-hf", add_eos_token=True)
# tokenizer.pad_token = tokenizer.eos_token
# tokenizer.pad_token_id = tokenizer.eos_token_id

data = load_dataset("json", data_files="data.json")

In [24]:
def generate_prompt(data_point):
    # sorry about the formatting disaster gotta move fast
    if data_point["instruction"]:
        return f"""Below is an instruction that describes a task, 
        paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{data_point["instruction"]}

### Input:
{data_point["input"]}

### Response:
{data_point["output"]}"""
#     else:
#         return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

# ### Instruction:
# {data_point["instruction"]}

# ### Response:
# {data_point["output"]}"""


# data = data.map(lambda data_point: {"prompt": tokenizer(generate_prompt(data_point))})

In [ ]:
import os

# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM,GenerationConfig
# from peft import PeftModel
# from transformers import LLaMATokenizer, LLaMAForCausalLM, GenerationConfig
from peft import prepare_model_for_int8_training, LoraConfig, get_peft_model,PeftModel

In [26]:
# Setting for A100 - For 3090 
MICRO_BATCH_SIZE = 8  # change to 4 for 3090
BATCH_SIZE = 128
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
EPOCHS = 2  # paper uses 3
LEARNING_RATE = 2e-5  # from the original paper
CUTOFF_LEN = 256  # 256 accounts for about 96% of the data
LORA_R = 4
LORA_ALPHA = 16
LORA_DROPOUT = 0.05

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "/content/open_llama_7b_preview_300bt/open_llama_7b_preview_300bt_transformers_weights",
    load_in_8bit=True,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained(
    "/content/open_llama_7b_preview_300bt/open_llama_7b_preview_300bt_transformers_weights", add_eos_token=True
)

model = prepare_model_for_int8_training(model)

In [28]:
config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)
tokenizer.pad_token_id = 0  # unk. we want this to be different from the eos token
data = load_dataset("json", data_files="data.json")


  0%|          | 0/1 [00:00<?, ?it/s]

In [29]:
model.print_trainable_parameters()

trainable params: 2097152 || all params: 6740512768 || trainable%: 0.03111264783824826


In [ ]:
# import torch
# torch.cuda.empty_cache()

In [ ]:
# import gc
# del variables
# gc.collect()

In [30]:

def generate_prompt(data_point):
    if data_point["input"]:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
{data_point["instruction"]}
### Input:
{data_point["input"]}
### Response:
{data_point["output"]}"""
#     else:
#         return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
# ### Instruction:
# {data_point["instruction"]}
# ### Response:
# {data_point["output"]}"""


data = data.shuffle().map(
    lambda data_point: tokenizer(
        generate_prompt(data_point),
        truncation=True,
        max_length=CUTOFF_LEN,
        padding="max_length",
    )
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=MICRO_BATCH_SIZE,
        gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
        # warmup_steps=100,
        num_train_epochs=10,
        learning_rate=LEARNING_RATE,
        fp16=True,
        logging_steps=1,
        output_dir="lora-alpaca",
        save_total_limit=3,
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False
trainer.train(resume_from_checkpoint=False)

model.save_pretrained("open_llama")

Map:   0%|          | 0/151 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:318: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
1,3.876600
2,3.869100
3,3.866400
4,3.861900
5,3.859300
6,3.857300
7,3.854100
8,3.851600
9,3.850100
10,3.850500


In [44]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
model.push_to_hub("gouravsinha/openllama-finance-ner", use_auth_token=True)

In [33]:
text = df.iloc[0]['sentence']

In [38]:
text = """India's financial system remains protected from "daunting" global challenges but there is a need for constant vigil, economic affairs secretary Ajay Seth said on Monday.
"""

In [39]:
PROMPT = f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
Your task is to extract the person's name from the sentence provided as input
along with their details mentioned below:
1. their designation, 
2. their companies ,
3. Number or percentage of shares bought
4. Type of shares
5. acquistions

The person should be either:
1. A person involved in sell or buying of any company's share at a large scale.
2. A person who is equivalent to CEO, chairman, board of directors,founder etc. 

If person name is not present then do not extract anything and respond 'None'
Extract the data in the list of python dictionaries format.
If the values are not available mark it as 'NA'.
### Input:
{text}
### Response:"""

In [40]:
inputs = tokenizer(
    PROMPT,
    return_tensors="pt",
)

In [41]:

input_ids = inputs["input_ids"].cuda()

generation_config = GenerationConfig(
    temperature=0.6,
    top_p=0.95,
    repetition_penalty=1.15,
)
print("Generating...")
generation_output = model.generate(
    input_ids=input_ids,
    generation_config=generation_config,
    return_dict_in_generate=True,
    output_scores=True,
    max_new_tokens=200,
)
for s in generation_output.sequences:
    print(tokenizer.decode(s))

Generating...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:318: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


<s> Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
Your task is to extract the person's name from the sentence provided as input
along with their details mentioned below:
1. their designation, 
2. their companies ,
3. Number or percentage of shares bought
4. Type of shares
5. acquistions

The person should be either:
1. A person involved in sell or buying of any company's share at a large scale.
2. A person who is equivalent to CEO, chairman, board of directors,founder etc. 

If person name is not present then do not extract anything and respond 'None'
Extract the data in the list of python dictionaries format.
If the values are not available mark it as 'NA'.
### Input:
India's financial system remains protected from "daunting" global challenges but there is a need for constant vigil, economic affairs secretary Ajay Seth said on Monday.

### Response:xThe person should be either:xA person involved in sell or bu

In [ ]:
text = df.iloc[1]['text']

In [ ]:
nlp = spacy.load("en_core_web_trf")
doc = nlp(text)
sent_lst = [str(sent) for sent in list(doc.sents)]

In [ ]:
text = "".join(sent_lst[0:4])

In [ ]:
output = chain.run(text)

## Text summarization

In [ ]:
from langchain import OpenAI, PromptTemplate, LLMChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate
from langchain.docstore.document import Document
from langchain.chains.summarize import load_summarize_chain

llm_s = OpenAI(temperature=0)

text_splitter = CharacterTextSplitter()

In [ ]:
text = df.iloc[1]['text']

In [ ]:
texts = text_splitter.split_text(text)
docs = [Document(page_content=t) for t in texts[:3]]
chain_s = load_summarize_chain(llm, chain_type="refine")

In [ ]:
from langchain.docstore.document import Document

docs = [Document(page_content=t) for t in texts[:3]]

In [ ]:
from langchain.chains.summarize import load_summarize_chain
chain_s = load_summarize_chain(llm, chain_type="refine")


In [ ]:
summary = chain_s.run(docs)

In [ ]:
print(chain.run(summary))

[{'Name': 'NA', 'Designation': 'NA', 'Company': 'Zee Entertainment Enterprises', 'Net Worth': 'NA', 'Shares bought': 'Rs 640 crore', 'Shares sold': 'NA', 'Type of Shares': 'NA', 'Acquisitions by the Company': 'NA'}, {'Name': 'NA', 'Designation': 'NA', 'Company': '360 ONE WAM', 'Net Worth': 'NA', 'Shares bought': 'NA', 'Shares sold': '49.96 lakh equity shares', 'Type of Shares': 'NA', 'Acquisitions by the Company': 'NA'}, {'Name': 'Sanjay Chamria', 'Designation': 'NA', 'Company': 'Poonawalla Fincorp', 'Net Worth': 'NA', 'Shares bought': 'NA', 'Shares sold': '83 lakh equity shares', 'Type of Shares': 'NA', 'Acquisitions by the Company': 'NA'}]
